<a href="https://colab.research.google.com/github/HanhaodiZhang/HATE/blob/main/augmentation_%3C%3E_5LOSS%2B10LOSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qq transformers

     |████████████████████████████████| 1.8MB 12.8MB/s 
     |████████████████████████████████| 890kB 47.2MB/s 
     |████████████████████████████████| 2.9MB 45.0MB/s 


In [2]:
import torch
import numpy as np
import pandas as pd

In [3]:
df_1 = pd.read_csv("2.csv", index_col='id')

In [4]:
possible_labels=df_1.subtask_a.unique()
label_dict={}
for index, possible_label in enumerate(possible_labels):
  label_dict[possible_label]=index

label_dict

{'NOT': 1, 'OFF': 0, nan: 2}

In [5]:
df_1['a_label']=df_1.subtask_a.replace(label_dict)
df_1.head()

,tweet,subtask_a,a_label
id,,,
864261.0,@USER She should ask a few native Americans wh...,OFF,0
864262.0,@USER She should whats a few native Americans ...,OFF,0
864263.0,@USER She should ask a few native Americans wh...,OFF,0
864261.0,@USER She should ask a few native Americans wh...,OFF,0
864262.0,@USER She should ask a few native Americans wh...,OFF,0


In [6]:
!pip install emoji --upgrade
!pip install wordsegment

     |████████████████████████████████| 51kB 6.0MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49717 sha256=c2c5201f81c434379332bbccb8be772c0d8f3dc8910b370478dc879faf965605
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji
     |████████████████████████████████| 4.8MB 13.0MB/s 


In [7]:
from wordsegment import load
load()

In [8]:
from wordsegment import segment
import emoji
import re
def clean(word):
  
  text = re.sub(r"can't", "can not ", word)
  text = re.sub(r"don't", "can not ", text)
  text = re.sub(r"\'s", "", word)
  text = re.sub(r"\'ve", "have", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"\'m", "am", text)
  text = re.sub(r"%", "percent", text) 
  return text 

def foremoji(text):
  text = re.sub(r"’", "'", text)
  text = re.sub(r":", " : ", text)
  text=emoji.demojize(text)
  text = text.split()
  new_text=[]
  for w in text: 
    w=clean(w)  
    w=re.sub(r":", " <emoji> ", w)
    w=re.sub(r"@USER","",w)
    w=re.sub(r"_"," ",w)
    if re.match("^#",w):
      w=" ".join(segment(w))
    new_text.append(w)
  text = " ".join(new_text)
  return text

def fake_foremoji(text):
  return '<fake> '+foremoji(text)
def real_foremoji(text):
  return '<real> '+foremoji(text)


In [9]:
df_1['emoji_tweet']=df_1.tweet.apply(fake_foremoji)

In [10]:
df_1

,tweet,subtask_a,a_label,emoji_tweet
id,,,,
864261.0,@USER She should ask a few native Americans wh...,OFF,0,<fake> She should ask a few native Americans ...
864262.0,@USER She should whats a few native Americans ...,OFF,0,<fake> She should whats a few native American...
864263.0,@USER She should ask a few native Americans wh...,OFF,0,<fake> She should ask a few native Americans ...
864261.0,@USER She should ask a few native Americans wh...,OFF,0,<fake> She should ask a few native Americans ...
864262.0,@USER She should ask a few native Americans wh...,OFF,0,<fake> She should ask a few native Americans ...
...,...,...,...,...
438512.0,@USER @USER @USER please get OFF first take! Y...,OFF,0,<fake> please get OFF first take! You yell ...
438513.0,@USER @USER @USER please get OFF first take! Y...,OFF,0,<fake> please get OFF first take! You yell ...
782741.0,@USER Doesn't surprise me. Texans think Libera...,OFF,0,<fake> Doesn't surprise me. Texans think Libe...


In [11]:
df_2 = pd.read_csv("olid-training.tsv", sep='\t', header=0, index_col='id')

In [12]:
df_2=df_2[['tweet','subtask_a']]

In [13]:
df_2

,tweet,subtask_a
id,,
86426,@USER She should ask a few native Americans wh...,OFF
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF
16820,Amazon is investigating Chinese employees who ...,NOT
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT
...,...,...
95338,@USER Sometimes I get strong vibes from people...,OFF
67210,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...,NOT
82921,@USER And why report this garbage. We don't g...,OFF


In [14]:
df_2['a_label']=df_2.subtask_a.replace(label_dict)
df_2.head()

,tweet,subtask_a,a_label
id,,,
86426,@USER She should ask a few native Americans wh...,OFF,0
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,0
16820,Amazon is investigating Chinese employees who ...,NOT,1
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,0
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,1


In [15]:
df_2['emoji_tweet']=df_2.tweet.apply(real_foremoji)

In [16]:
df_2

,tweet,subtask_a,a_label,emoji_tweet
id,,,,
86426,@USER She should ask a few native Americans wh...,OFF,0,<real> She should ask a few native Americans ...
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,0,<real> Go home you are drunk!!! maga trump2...
16820,Amazon is investigating Chinese employees who ...,NOT,1,<real> Amazon is investigating Chinese employe...
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,0,"<real> Someone shouldhaveTaken"" this piece of..."
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,1,<real> Obama wanted liberals &amp; illegals ...
...,...,...,...,...
95338,@USER Sometimes I get strong vibes from people...,OFF,0,<real> Sometimes I get strong vibes from peop...
67210,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...,NOT,1,<real> Benidorm <emoji> white heavy check mar...
82921,@USER And why report this garbage. We don't g...,OFF,0,<real> And why report this garbage. We don't ...


In [17]:
df = pd.concat( [df_1, df_2], axis=0 )

In [18]:
df

,tweet,subtask_a,a_label,emoji_tweet
id,,,,
864261.0,@USER She should ask a few native Americans wh...,OFF,0,<fake> She should ask a few native Americans ...
864262.0,@USER She should whats a few native Americans ...,OFF,0,<fake> She should whats a few native American...
864263.0,@USER She should ask a few native Americans wh...,OFF,0,<fake> She should ask a few native Americans ...
864261.0,@USER She should ask a few native Americans wh...,OFF,0,<fake> She should ask a few native Americans ...
864262.0,@USER She should ask a few native Americans wh...,OFF,0,<fake> She should ask a few native Americans ...
...,...,...,...,...
95338.0,@USER Sometimes I get strong vibes from people...,OFF,0,<real> Sometimes I get strong vibes from peop...
67210.0,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...,NOT,1,<real> Benidorm <emoji> white heavy check mar...
82921.0,@USER And why report this garbage. We don't g...,OFF,0,<real> And why report this garbage. We don't ...


In [20]:
df_1['data_type']=['train']*df_1.shape[0]
df_2['data_type']=['train']*df_2.shape[0]

In [21]:
from transformers import BertTokenizer

PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [22]:
from transformers import BertForSequenceClassification

model=BertForSequenceClassification.from_pretrained(
    PRE_TRAINED_MODEL_NAME,
    num_labels=len(label_dict),
    output_attentions=False,
    output_hidden_states=False,
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [23]:
tokenizer.add_special_tokens({'additional_special_tokens':["<emoji>","<real>","<fake>"]})
model.resize_token_embeddings(len(tokenizer))

Embedding(28999, 768)

In [24]:
dfval = pd.read_csv("olid-training.tsv", sep='\t', header=0, index_col='id')
dfval=dfval[['tweet','subtask_a']]
dfval.head()

,tweet,subtask_a
id,,
86426,@USER She should ask a few native Americans wh...,OFF
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF
16820,Amazon is investigating Chinese employees who ...,NOT
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT


In [25]:
dfval['a_label']=dfval.subtask_a.replace(label_dict)
dfval.head()

,tweet,subtask_a,a_label
id,,,
86426,@USER She should ask a few native Americans wh...,OFF,0
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,0
16820,Amazon is investigating Chinese employees who ...,NOT,1
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,0
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,1


In [26]:
dfval['emoji_tweet']=dfval.tweet.apply(real_foremoji)

In [27]:
from sklearn.model_selection import train_test_split
RANDOM_SEED=17

X_train,X_val,y_train,y_val = train_test_split(
  dfval.index.values,
  dfval.a_label.values,
  test_size=0.5,
  random_state=RANDOM_SEED,
  # stratify=df.a_label.values
)

In [28]:
dfval['data_type']=['not_set']*dfval.shape[0]

dfval.loc[X_train,'data_type']='train'
dfval.loc[X_val,'data_type']='val'
dfval.head()

,tweet,subtask_a,a_label,emoji_tweet,data_type
id,,,,,
86426,@USER She should ask a few native Americans wh...,OFF,0,<real> She should ask a few native Americans ...,val
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,0,<real> Go home you are drunk!!! maga trump2...,train
16820,Amazon is investigating Chinese employees who ...,NOT,1,<real> Amazon is investigating Chinese employe...,train
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,0,"<real> Someone shouldhaveTaken"" this piece of...",train
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,1,<real> Obama wanted liberals &amp; illegals ...,train


In [29]:
encoder_data_train_real=tokenizer.batch_encode_plus(
    df_2[df_2.data_type=='train'].emoji_tweet.values,
    add_special_tokens=True,
    return_attention_mask=True,
    truncation=True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)
encoder_data_train_fake=tokenizer.batch_encode_plus(
    df_1[df_1.data_type=='train'].emoji_tweet.values,
    add_special_tokens=True,
    return_attention_mask=True,
    truncation=True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)
encoder_data_val=tokenizer.batch_encode_plus(
    dfval[dfval.data_type=='val'].emoji_tweet.values,
    add_special_tokens=True,
    return_attention_mask=True,
    truncation =True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)


In [30]:
inputs_ids_train_real=encoder_data_train_real['input_ids']
attentions_masks_train_real=encoder_data_train_real['attention_mask']
labels_train_real=torch.tensor(df_2[df_2.data_type=='train'].a_label.values)


inputs_ids_train_fake=encoder_data_train_fake['input_ids']
attentions_masks_train_fake=encoder_data_train_fake['attention_mask']
labels_train_fake=torch.tensor(df_1[df_1.data_type=='train'].a_label.values)

inputs_ids_val=encoder_data_val['input_ids']
attentions_masks_val=encoder_data_val['attention_mask']
labels_val=torch.tensor(dfval[dfval.data_type=='val'].a_label.values)

In [31]:
from torch.utils.data import TensorDataset
dataset_train_real=TensorDataset(inputs_ids_train_real,
              attentions_masks_train_real, labels_train_real)
dataset_train_fake=TensorDataset(inputs_ids_train_fake,
              attentions_masks_train_fake, labels_train_fake)
dataset_val=TensorDataset(inputs_ids_val,
              attentions_masks_val, labels_val)

In [47]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

BATCH_SIZE=16

dataloader_train_real=DataLoader(
    dataset_train_real,
    sampler=RandomSampler(dataset_train_real),
    batch_size=BATCH_SIZE
)

dataloader_train_fake=DataLoader(
    dataset_train_fake,
    sampler=RandomSampler(dataset_train_fake),
    batch_size=BATCH_SIZE
)


dataloader_val=DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=BATCH_SIZE
)

In [48]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
epochs = 4

scheduler = get_linear_schedule_with_warmup(optimizer, 
                        num_warmup_steps=0,
                        num_training_steps=len(dataloader_train_fake)*epochs)

In [49]:

import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def classification_report_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return classification_report(labels_flat, preds_flat, digits=5)

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [50]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':batch[0],
              'attention_mask':batch[1],
              'labels':batch[2],
              }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [90]:
epochs=3

In [43]:
loarder=zip(cycle(dataloader_train_real),dataloader_train_fake)

In [52]:

for (batch_real, batch_fake) in enumerate(zip(cycle(dataloader_train_real),dataloader_train_fake)):
  pass

In [91]:
from tqdm.notebook import tqdm
from itertools import cycle
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train_fake, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)

    for i,(batch_real, batch_fake) in enumerate(zip(cycle(dataloader_train_real),progress_bar)):
        
        model.zero_grad()
        
        batch_real = tuple(b.to(device) for b in batch_real)
        batch_fake = tuple(b.to(device) for b in batch_fake) 

        inputs_real = {'input_ids':      batch_real[0],
                  'attention_mask': batch_real[1],
                  'labels':         batch_real[2],
                 }       
        inputs_fake = {'input_ids':      batch_fake[0],
                  'attention_mask': batch_fake[1],
                  'labels':         batch_fake[2],
                 }       

        outputs_real = model(**inputs_real)
        outputs_fake = model(**inputs_fake)
               
        loss = 5*outputs_fake[0]+10*outputs_real[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch_fake))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train_fake)             
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 1.5725444776488535
Validation loss: 0.06221039477648024
F1 Score (Weighted): 0.9839568900620638



Epoch 2
Training loss: 1.539846583635333
Validation loss: 0.06194900095593274
F1 Score (Weighted): 0.9839568900620638



Epoch 3
Training loss: 1.5482681435948036
Validation loss: 0.06194891139599472
F1 Score (Weighted): 0.9839568900620638



In [57]:
df_sweet = pd.read_csv("testset-levela.tsv", sep='\t', header=0, index_col='id')
df_sweet.head()

,tweet
id,
15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...
27014,"#ConstitutionDay is revered by Conservatives, ..."
30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...
13876,#Watching #Boomer getting the news that she is...
60133,#NoPasaran: Unity demo to oppose the far-right...


In [58]:
df_l = pd.read_csv("labels-levela.csv",header=None,names=['id','subtask_a'],index_col='id')

In [59]:
test = pd.merge(df_sweet, df_l, on=['id'])

In [60]:
test['data_type']=['test']*test.shape[0]

In [61]:
test['a_label']=test.subtask_a.replace(label_dict)
test['emoji_tweet']=test.tweet.apply(real_foremoji)

encoder_data_test=tokenizer.batch_encode_plus(
    test[test.data_type=='test'].emoji_tweet.values,
    add_special_tokens=True,
    return_attention_mask=True,
    truncation=True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)
inputs_ids_test=encoder_data_test['input_ids']
attentions_masks_test=encoder_data_test['attention_mask']
labels_test=torch.tensor(test[test.data_type=='test'].a_label.values)
dataset_test=TensorDataset(inputs_ids_test,
              attentions_masks_test, labels_test)
dataloader_test=DataLoader(
    dataset_test,
    sampler=RandomSampler(dataset_test),
    batch_size=4
)

In [100]:
model.load_state_dict(torch.load('finetuned_BERT_epoch_2.model', map_location=torch.device('cuda')))

<All keys matched successfully>

In [101]:
_, predictions, true_test = evaluate(dataloader_test)

In [102]:
accuracy_per_class(predictions, true_test)

Class: OFF
Accuracy: 148/240

Class: NOT
Accuracy: 551/620



In [103]:
print(classification_report_func(predictions, true_test))

              precision    recall  f1-score   support

           0    0.68203   0.61667   0.64770       240
           1    0.85692   0.88871   0.87253       620

    accuracy                        0.81279       860
   macro avg    0.76947   0.75269   0.76011       860
weighted avg    0.80811   0.81279   0.80978       860

